In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime, timedelta
from math import sqrt
import  pylab as pl
import altair as alt
import io
import dropbox


D = 400
rf = 0 #Risk free return

tk = '5-UkyaE_0XoAAAAAAAAAAb-BCtdL-qKmMTbSNOKdSSXwxA5hFBjrERMGyHcjInpW'
DBX = dropbox.Dropbox(tk)

date_D_days_ago = datetime.now() - timedelta(days=D)
now = datetime.now()

start_date = date_D_days_ago.strftime('%F')
end_date = now.strftime('%F')

#Fortmat output
pct = lambda x: '{:.2%}'.format(x)
dig = lambda x: '{:.2f}'.format(x)
to_float = lambda x: float(x.strip('%'))/100
custom_date_parser = lambda x: datetime.strptime(x,"%Y-%m-%d")


#Get Data from Yahoo
def StockData(ticker, start_d, end_d):
    data = web.get_data_yahoo(ticker, start = start_d, end = end_d)
    price =  pd.DataFrame(data['Adj Close'])
    
    #volume = pd.DataFrame(data['Volume'])
    return price

#Get price from listed US stocks 
def PriceDataSet():
    exchange = ['nasdaq','nyse','amex']

    df_nasdaq = GetFile(exchange[0])
    df_nyse = GetFile(exchange[1])
    df_amex = GetFile(exchange[2])
    df = df_nasdaq.append(df_nyse).append(df_amex)  

    df.index = df.index.astype('str')

    # Replace tickers with "." with no info
    df = df[df.index.str.contains("\.",) == False]
    df = df[df.index.str.contains("\/",) == False]

    ticker = np.array(df.index)

    price1 = StockData(ticker[:1000], start_date, end_date)
    price2 = StockData(ticker[1001:], start_date, end_date)
    price =pd.concat([price1, price2], axis=1)
    price = price.dropna(axis=1, how='all')
    
    #price.to_csv('../data/price.csv', index = 'date')
    return PostFile(price,'price')
    
# Max Sharpe with Min correlation
def MaxSharpe_MinCorr(new_df, sharpe, asset, num):
    sharpe = sharpe.drop(asset)    
    max_sharpe = sharpe.sort_values(ascending = False).head(num)
    list_max = np.array(max_sharpe.index.values)
    porfolio_A = new_df[list_max].corrwith(new_df[asset]).abs().sort_values(ascending = True).head(9)  
    porfolio_A = pd.Series(porfolio_A.index.values)
    return porfolio_A

# Min correlation with Max Sharpe
def MinCorr_MaxSharpe(new_df, asset, num):
    porfolio_B = new_df.corrwith(new_df[asset]).abs().sort_values(ascending = True).head(num)
    new_df = new_df[porfolio_B.index.values]
    f_sharpe = (250**0.5)*(new_df.mean()/new_df.std())    
    f_sharpe.sort_values(ascending = False)
    max_sharpe = f_sharpe.head(9)
    porfolio_B = pd.Series(max_sharpe.index.values)
    return porfolio_B




#Download files from DBX
def GetFile(filename):    
    _, read = DBX.files_download("/data/"+filename+".csv")
    
    with io.BytesIO(read.content) as stream:
        if filename == 'price':
            df = pd.read_csv(stream, index_col=['Date'], parse_dates=['Date'], date_parser=custom_date_parser)
        else:
            df = pd.read_csv(stream, index_col=0)
    return df


#Upload files to DBX
def PostFile(df,name):    
    data = df.to_csv(index=True) # The index parameter is optional
    db_bytes = bytes(data, 'utf8')
 
    DBX.files_upload(db_bytes,"/data/"+name+".csv", mode=dropbox.files.WriteMode.overwrite)
    return print(name+' uploaded on '+str(now))


# Pofolio Optimization and Efficient Frontier (TO REVIEW)
def P_Optimization(df):
    ind_er = df.pct_change().apply(lambda x: np.log(1+x)).mean().apply(lambda x: x*250)
    cov_matrix = df.pct_change().apply(lambda x: np.log(1+x)).cov()
    #corr_matrix = df.pct_change().apply(lambda x: np.log(1+x)).corr()
    ann_sd = df.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))

    p_ret = [] # Define an empty array for portfolio returns
    p_vol = [] # Define an empty array for portfolio volatility
    p_weights = [] # Define an empty array for asset weights

    num_assets = len(df.columns)
    num_portfolios = 10000
    
    for portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights = weights/np.sum(weights)
        weights= weights.round(2)
        p_weights.append(weights)
        returns = np.dot(weights, ind_er) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
        p_ret.append(returns)
        var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
        sd = np.sqrt(var) # Daily standard deviation
        ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
        p_vol.append(ann_sd)
    
    data = {'Returns':p_ret, 'Volatility':p_vol}

    for counter, symbol in enumerate(df.columns.tolist()):
        #print(counter, symbol)
        data[symbol+' weight'] = [w[counter] for w in p_weights]

    portfolios  = pd.DataFrame(data) #Dataframe of the 10000 portfolios created
    return portfolios

def Plot_P_Optimization(porfolios,price):
        min_vol_port = portfolios.loc[portfolios['Volatility'].idxmin()]
        optimal_risky_port = portfolios.loc[((portfolios['Returns']- rf)/portfolios['Volatility']).idxmax()]
        ann_mean = price.pct_change().apply(lambda x: np.log(1+x)).mean().apply(lambda x: x*250)
        ann_std = price.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))
              
        # Plotting optimal portfolio
    #    portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])
        plt.subplots(figsize=(10, 10))
        plt.scatter(portfolios['Volatility'], portfolios['Returns'], marker='o', s=10, alpha=0.3)
        plt.xlabel('Volatility', fontsize=20)
        plt.ylabel('Returns', fontsize=20)
        plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=400, label='Minimum volatility')
        plt.legend(loc='upper left', fontsize=12)
        plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=400, label='Maximum Sharpe ratio')
        plt.legend(loc='best', fontsize=12)
                
        #Plotting individual assets
        for i in price.columns:
            plt.scatter(ann_std[i], ann_mean[i], color='b' , marker='.', s=500)
            plt.annotate('   '+i, (ann_std[i], ann_mean[i]), ha='left', va='center')
    
         #Saving plots to use them later    
        plt.savefig('data/img/'+s+'_optimal_porfolio.png')
        plt.cla()
        plt.close('all')
        #return 

#K-Means Clustering
def Clustering(ann_mean, ann_std):
        ret_var = pd.concat([ann_mean, ann_std], axis = 1).dropna()
        ret_var.columns = ["Return","Volatility"]

        X =  ret_var.values #Converting ret_var into nummpy arraysse = []for k in range(2,15):
        
        
        sse = []
        
        for k in range(2,15):
            kmeans = KMeans(n_clusters = k)
            kmeans.fit(X)
            sse.append(kmeans.inertia_) #SSE for each n_clusterspl.plot(range(2,15), sse)
        pl.plot(range(2,15), sse)      
        pl.title("Elbow Curve")
        pl.show()
        
        
        df = pd.DataFrame(X, index=ret_var.index, columns=['X','Y'])
        alt.Chart(df).mark_point().encode(x='X',y='Y',shape='kmeans.labels_')
        #pl.scatter(X[:,1],X[:,0], c = kmeans.labels_, cmap ="rainbow")
        #plt.scatter(centroids[:,1],centroids[:,0], marker = 'x', color = 'b', label = 'Centroids')
        kmeans = KMeans(n_clusters = 5).fit(X)
        centroids = kmeans.cluster_centers_
        cluster_labels = pd.DataFrame(kmeans.labels_, index=ret_var.index, columns=['Clustering'])
        #tupper = pd.concat([tupper, cluster_labels],axis = 1)
        print ('Builing Clustering with the ML Library K-Means') 
        return cluster_labels
    
    
#Performace for each stock
def Performance(p):
    window = ['D','W','M','3M','6M','A']
    perform = pd.DataFrame(0, index = p.columns, columns=window)

    for i in window:    
        df = p.resample(i).last().pct_change().tail(2)
        perform[i] = df.iloc[0,:].apply(pct) 
    
    df = p.resample('Y').last().pct_change().tail(1)
    perform['YTD'] = df.iloc[0,:].apply(pct)
    
    return perform


# Read industry and sectors of stock market
#info = pd.read_csv('../data/S&P500-Info.csv', index_col=['Symbol'])
#info = pd.DataFrame(info)

PriceDataSet()

#custom_date_parser = lambda x: datetime.strptime(x,"%Y-%m-%d")

price = GetFile('price')
#price = pd.DataFrame(price, index_col=['Date'], parse_dates=['Date'], date_parser=custom_date_parser)
#price = price.index_col(['Date']).date_parser(custom_date_parser)
#price = pd.read_csv('../data/price.csv', index_col=['Date'], parse_dates=['Date'], date_parser=custom_date_parser)
#price = pd.DataFrame(price)

# Read  Tickers
tickers = price.columns



# Expected annualized Return, Volatility, Correlation and Sharpe
ann_mean = price.pct_change().apply(lambda x: np.log(1+x)).mean().apply(lambda x: x*252)
ann_std = price.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(252))
corr = price.pct_change().apply(lambda x: np.log(1+x)).corr()
cov  = price.pct_change().apply(lambda x: np.log(1+x)).cov()
Sharpe = (ann_mean - rf)/ann_std
perform_df = Performance(price)



# Building Tupperware
tupper = pd.DataFrame(ann_mean, columns=['Return'], index = tickers)
tupper.index.name = 'ticker'
tupper ['Volatility'] = pd.DataFrame(ann_std, columns=['Volatility'])
tupper ['Sharpe'] = pd.DataFrame(Sharpe, columns=['Sharpe'])
tupper ['Min_Corr'] = pd.DataFrame(corr.abs().idxmin(), columns=['Min_Corr'])
tupper ['Corr_value'] = pd.DataFrame(corr.min(), columns=['Corr_Value'])


#cluster_labels = Clustering(ann_mean, ann_std)
tupper = pd.concat([tupper, perform_df],axis = 1)

#pd.merge(df1, df2, left_index=True, right_index=True)


tupper = pd.merge(tupper, df[['Name','Country','Sector','Industry','IPO Year','Market Cap']], left_index=True, right_index=True)

#Save file to DBX
PostFile(tupper,'tupper')

#tupper.to_csv('../data/tupper_'+end_date+'.csv')

%time

/Users/datapro/opt/anaconda3/lib/python3.8/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'HMSY', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/datapro/opt/anaconda3/lib/python3.8/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'PFPT', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/datapro/opt/anaconda3/lib/python3.8/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'TTD', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/datapro/opt/anaconda3/lib/python3.8/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'BRO', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/datapro/opt/anaconda3/lib/python3.8/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'BXP', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWa

price uploaded on 2021-02-09 16:14:25.830308


NameError: name 'df' is not defined

In [51]:
tupper

,Return,Volatility,Sharpe,Min_Corr,Corr_value,D,W,M,3M,6M,A,YTD,Name,Country,Sector,Industry,IPO Year,Market Cap
AAL,-0.422700,0.983255,-0.429899,MDB,-1.000000,-1.14%,8.53%,8.88%,52.22%,54.41%,nan%,9.00%,American Airlines Group Inc. Common Stock,United States,Transportation,Air Freight/Delivery Services,NaN,1.081374e+10
AAON,0.405802,0.493774,0.821836,AEM,-1.000000,1.76%,1.73%,11.06%,27.05%,25.25%,nan%,16.52%,AAON Inc. Common Stock,United States,Capital Goods,Industrial Machinery/Components,NaN,4.015850e+09
AAPL,0.562400,0.459595,1.223685,LSXMB,-1.000000,2.58%,-5.11%,-0.55%,21.43%,24.62%,nan%,3.22%,Apple Inc. Common Stock,United States,Technology,Computer Manufacturing,1980.0,2.381977e+12
ABCB,0.055616,0.722794,0.076946,QS,-1.000000,4.53%,-11.76%,2.73%,34.02%,71.26%,nan%,15.58%,Ameris Bancorp Common Stock,United States,Finance,Major Banks,1994.0,3.050785e+09
ABCL,-1.602680,0.923726,-1.735017,AFRM,-0.441451,-4.06%,4.61%,31.29%,nan%,nan%,nan%,15.68%,AbCellera Biologics Inc. Common Shares,Canada,Health Care,Major Pharmaceuticals,2020.0,1.253877e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IMO,-0.228748,0.697958,-0.327739,BNR,-1.000000,2.02%,-5.61%,-0.05%,44.07%,24.18%,nan%,6.31%,Imperial Oil Limited Common Stock,Canada,Energy,Integrated oil Companies,NaN,1.486505e+10
LNG,0.048788,0.534203,0.091328,CNXC,-0.670930,-0.84%,-4.05%,5.50%,32.30%,27.99%,nan%,10.59%,Cheniere Energy Inc. Common Stock,United States,Public Utilities,Oil/Gas Transmission,NaN,1.637258e+10
NG,0.051567,0.700227,0.073643,EPR,-0.273934,-0.33%,0.22%,-5.69%,-11.97%,0.00%,nan%,-5.17%,Novagold Resources Inc.,Canada,Basic Industries,Precious Metals,NaN,3.038210e+09
PRK,0.142173,0.564237,0.251974,GFI,-1.000000,3.75%,-5.02%,2.86%,19.01%,28.70%,nan%,8.19%,Park National Corporation Common Stock,United States,Finance,Major Banks,NaN,2.000405e+09


In [ ]:

portfolio_df= pd.DataFrame(0, columns = ['a0','a1','a2','a3','a4','a5','a6','a7','a8','a9'], index = tickers)                                            
portfolio_df['a0'] = portfolio_df.index.values

min_vol_port_df = pd.DataFrame(0, columns = ['mv_Returns','mv_Volatility','mv_w0','mv_w1','mv_w2','mv_w3','mv_w4','mv_w5','mv_w6','mv_w7','mv_w8','mv_w9'], index = sp500)
optimal_risky_port_df = pd.DataFrame(0, columns = ['or_Returns','or_Volatility','or_w0','or_w1','or_w2','or_w3','or_w4','or_w5','or_w6','or_w7','or_w8','or_w9'], index = sp500)
    


#Building porfolios 
for r in portfolio_df.index.values:
    Test_A = MaxSharpe_MinCorr(price, Sharpe, r, 100)
    for c in range(9):
        portfolio_df.loc[r,'a'+str(c+1)] = Test_A[c]
         
ti = 0
    
for s in portfolio_df.index.values:
    p = price[portfolio_df.loc[s,:]]
    ti = ti +1
    portfolios = P_Optimization (p)
    min_vol_port = portfolios.loc[portfolios['Volatility'].idxmin()]
    optimal_risky_port = portfolios.loc[((portfolios['Returns']- rf)/portfolios['Volatility']).idxmax()]
    print('Porfolio '+str(ti)+' optimized.')
              
    # Plotting optimal portfolio
    #portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])
    #Plot_P_Optimization(portfolios,p)
        
    #Saving optimal porfolios weights in a DataFrame
    min_vol_port_df.loc[s,:]  = min_vol_port.values
    optimal_risky_port_df.loc[s,:] = optimal_risky_port.values
    
   
print('\n Porfolio creation ='+str(ti)) 

#Saving data
tupper = pd.concat([tupper,info,portfolio_df,min_vol_port_df,optimal_risky_port_df], axis=1)
tupper.to_csv('data/tupper_'+end_date+'.csv')

print ('Done! /n Tupperware Data created on '+end_date)

In [ ]:
df = price[['TSLA','CSCO']]

p = Performance(df)

In [ ]:
p.columns.name = 'window'
p = p.reset_index()
p

In [ ]:

ret_var = pd.concat([ann_mean, ann_std], axis = 1).dropna()
ret_var.columns = ["Return","Volatility"]

X =  ret_var.values #Converting ret_var into nummpy arraysse = []for k in range(2,15):
df = pd.DataFrame(X, index=ret_var.index, columns=['Return','Volatility'])
 
kmeans = KMeans(n_clusters = 5).fit(X)
centroids = kmeans.cluster_centers_
cluster_labels = pd.DataFrame(kmeans.labels_, index=ret_var.index, columns=['Clustering'])
df = pd.concat([df, cluster_labels],axis = 1)

In [ ]:
df = df.reset_index()
alt.Chart(df).mark_point().encode(x='Volatility',y='Return',color='Clustering:N', tooltip=['ticker:N', 'Volatility:N','Return:N'])



In [ ]:
fecha = lambda x: x.strftime('%F')

lista = ['TSLA','CSCO']

source = price[lista]
source = source.reset_index()
source = source.melt('Date', var_name='ticker', value_name='price')
source['price'] = source['price'].apply(dig)
source['Date'] = source['Date'].apply(fecha)


In [ ]:
fecha = lambda x: x.strftime('%F')

def plot_Performance(df):
    df = df / df.iloc[0]
    df = df.reset_index()
    df = df.melt('Date', var_name='ticker', value_name='price')
    df['price'] = df['price'].apply(dig)
    df['Date'] = df['Date'].apply(fecha)
    pic = alt.Chart(df).mark_line().encode(
        x='Date:T',
        y='price:Q',
        color='ticker:N', tooltip=['ticker:N', 'Date:N','price:N']
    )
    return pic

In [12]:
def StockData(ticker, start_d, end_d):
    data = web.get_data_yahoo(ticker, start = start_d, end = end_d)
    price =  pd.DataFrame(data['Adj Close'])
    
    #volume = pd.DataFrame(data['Volume'])
    return price



exchange = ['nasdaq','nyse','amex']



df_nasdaq = GetFile(exchange[0])
df_nyse = GetFile(exchange[1])
df_amex = GetFile(exchange[2])
df = df_nasdaq.append(df_nyse).append(df_amex)  


df.index = df.index.astype('str')

# Replace tickers with "." with no info
df = df[df.index.str.contains("\.",) == False]
df = df[df.index.str.contains("\/",) == False]

ticker = np.array(df.index)

In [9]:
price1 = StockData(ticker[:1000], start_date, end_date)
price2 = StockData(ticker[1001:], start_date, end_date)
price =pd.concat([price1, price2], axis=1)
price = price.dropna(axis=1, how='all')
price.to_csv('../data/price.csv', index = 'date')

In [16]:
df

,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
Symbol,,,,,,,,,,
AAL,American Airlines Group Inc. Common Stock,$17.40,-0.20,-1.136%,1.081374e+10,United States,NaN,37606820,Transportation,Air Freight/Delivery Services
AAON,AAON Inc. Common Stock,$76.87,1.33,1.761%,4.015850e+09,United States,NaN,112730,Capital Goods,Industrial Machinery/Components
AAPL,Apple Inc. Common Stock,$137.39,3.45,2.576%,2.381977e+12,United States,1980.0,84183061,Technology,Computer Manufacturing
ABCB,Ameris Bancorp Common Stock,$43.87,1.90,4.527%,3.050785e+09,United States,1994.0,395755,Finance,Major Banks
ABCL,AbCellera Biologics Inc. Common Shares,$46.58,-1.97,-4.058%,1.253877e+10,Canada,2020.0,373990,Health Care,Major Pharmaceuticals
...,...,...,...,...,...,...,...,...,...,...
IMO,Imperial Oil Limited Common Stock,$20.25,0.40,2.015%,1.486505e+10,Canada,NaN,944879,Energy,Integrated oil Companies
LNG,Cheniere Energy Inc. Common Stock,$64.90,-0.55,-0.84%,1.637258e+10,United States,NaN,879628,Public Utilities,Oil/Gas Transmission
NG,Novagold Resources Inc.,$9.17,-0.03,-0.326%,3.038210e+09,Canada,NaN,812928,Basic Industries,Precious Metals


In [19]:
pd.merge(tupper, df[['Name','Country','Sector','Industry']], left_index=True, right_index=True)

Return                                       0.332633
Volatility                                    0.50285
Sharpe                                       0.661495
Min_Corr                                         OCDX
Corr_value                                         -1
D                                               1.35%
W                                              -0.09%
M                                               1.36%
3M                                             -2.89%
6M                                             15.76%
A                                                nan%
YTD                                             7.35%
Name                  Salesforce.com Inc Common Stock
Country                                 United States
Sector                                     Technology
Industry      Computer Software: Prepackaged Software
Name: CRM, dtype: object

In [32]:
def PostFile(df,name):    
    data = df.to_csv(index=True) # The index parameter is optional
    db_bytes = bytes(data, 'utf8')
 
    
    DBX.files_upload(db_bytes,"/data/"+name+".csv", mode=dropbox.files.WriteMode.overwrite)
    return print(name+' uploaded on '+str(now))



custom_date_parser = lambda x: datetime.strptime(x,"%Y-%m-%d")

price = pd.read_csv('../data/price.csv', index_col=['Date'], parse_dates=['Date'], date_parser=custom_date_parser)
price = pd.DataFrame(price)

file = "price2"
PostFile(price,file)

price2 Uploaded on 2021-02-08 12:04:06.409496


In [ ]:
price.